In [124]:
import numpy as np

In [125]:
class LinearRegression:
    def __init__(self, dim):
        self.dim = max(1, dim)
        self.weights = np.zeros((1+dim,1))

    def X_T(self,X):
        num_examples = X.shape[0]
        real_X = np.c_[np.ones(num_examples), X]
        return real_X

    def train(self,X,Y):
        real_X = self.X_T(X)
        pinv_X = np.linalg.pinv(real_X)
        self.weights = np.dot(pinv_X,Y)
        
    def predict(self,X):
        real_X = self.X_T(X)
        h = np.matmul(real_X, self.weights)
        return h

In [126]:
class PLA:
    def __init__(self, dim):
        self.dim = dim
        self.weights = np.zeros(1+dim)

    def train(self,x,y):
        guess = self.predict(x)
        realx = np.append([1],x[:self.dim])
        agreed = guess == y
        if not agreed:
            self.weights = self.weights + np.multiply(y,realx)
        return agreed
    
    def predict(self,x):
        realx = np.append([1],x[:self.dim])
        h = np.dot(self.weights, realx)
        return np.sign(h)

In [127]:
class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        diff = np.subtract(p2, p1)
        if diff[0] <= 0.00001:
            self.m = None
            self.vert = True
        else:
            self.m = diff[1]/diff[0]
            self.vert = False
        if not self.vert:
            self.b = ((-1 * p1[1])/self.m) + p1[0]
            
    def label(self,testpt):
        if self.vert == False:
            line_y = self.m*testpt[0] + self.b
            diff = testpt[1] - line_y
        else:
            line_x = self.p1[0]
            diff = testpt[0] - line_x
        return np.sign(diff)

In [128]:
class LR_EXP:        
    def __init__(self, numpoints):
        self.n = numpoints
        self.lr = LinearRegression(2)
        self.points = np.random.uniform(-1.0,1.0,(self.n, 2))
        self.p12 = [np.random.uniform(-1.0,1.0,2) for x in range(2)]
        self.target = Line(self.p12[0],self.p12[1])
        while self.p12[0][0] == self.p12[1][0] and self.p12[0][1] == self.p12[1][1]:
            p12 = [np.random.uniform(-1.0,1.0,2) for x in range(2)]       
        self.labels = np.array([self.target.label(x) for x in self.points])
        
    def train(self):
        self.lr.train(self.points, self.labels)
    #E_in(w) = (1/N)*L2norm(X*w-y)
    
    def Eout_points(self, numpoints):
        self.n = numpoints
        self.points = np.random.uniform(-1.0,1.0,(self.n, 2))
        self.labels = np.array([self.target.label(x) for x in self.points])
    
    def e_in(self):
        xw = self.lr.predict(self.points)
        xw = np.sign(xw)
        mydiff = np.not_equal(xw, self.labels)
        e_in = np.mean(mydiff)
        return e_in

In [129]:
class PLA_EXP:        
    def __init__(self, numpoints):
        self.n = numpoints
        self.points = [np.random.uniform(-1.0,1.0,2) for x in range(numpoints)]
        p = [np.random.uniform(-1.0,1.0,2) for x in range(2)]
        while p[0][0] == p[1][0] and p[0][1] == p[1][1]:
            p = [np.random.uniform(-1.0,1.0,2) for x in range(2)]
        self.target = Line(p[0],p[1])
        self.PLA = PLA(2)

    def agreement(self,point):
        h_p = self.PLA.predict(point)
        actval = self.target.label(point)
        return h_p == actval

    def iteration(self):
        iters = 0
        testidx = 0
        while True:
            actval = self.target.label(self.points[testidx])
            success = self.PLA.train(self.points[testidx],actval)
            if success:
                allgood = True
                for i in range(self.n):
                    agree = self.agreement(self.points[i])
                    if not agree:
                        allgood = False
                        break
                if allgood:
                    break
            else:
                iters = iters + 1
            testidx = int(np.random.uniform(0, self.n-0.5))
        return iters

In [140]:
def prob5to7(num_exp):
    nin = 100
    nout = 1000
    npla = 10
    g_ein = np.array([])
    g_eout = np.array([])
    g_iters = np.array([])
    
    for i in range(num_exp):
        h_lr = LR_EXP(nin)
        h_lr.train()
        
        h_ein = h_lr.e_in()
        g_ein = np.concatenate((g_ein,[h_ein]))
        
        h_lr.Eout_points(nout)
        h_eout = h_lr.e_in()
        g_eout = np.concatenate((g_eout,[h_eout]))
        
        h_pla = PLA_EXP(npla)
        h_pla.target = h_lr.target
        h_pla.PLA.weights = h_lr.lr.weights.T
        h_iter = h_pla.iteration()
        g_iters = np.concatenate((g_iters,[h_iter]))
        
    ein_avg = np.average(g_ein)
    eout_avg = np.average(g_eout)
    iters_avg = np.average(g_iters)
    
    print("e_in average:",  ein_avg)
    print("e_out average:",  eout_avg)
    print("perceptron convergence average:",  iters_avg)

In [142]:
prob5to7(100)

e_in average: 0.0286
e_out average: 0.036950000000000004
perceptron convergence average: 2.24
